In [1]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import numpy

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'

sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util
import re

In [8]:

def summarize_allele_info(observedMaf, contextMaf, pentasToLookAt):

    listOfDicts = []
    for penta in pentasToLookAt:
        pentasPossible = contextMaf[contextMaf['pentaChange'] == penta]
        possiblePentaAlleles = set(pentasPossible['allele'])
        nPossiblePentas = pentasPossible.shape[0]
        if nPossiblePentas > 0:
            realMafPenta = observedMaf[observedMaf['pentaChange'] == penta]
            observedAlleles = set(realMafPenta['allele'])
            neverObservedAlleles = possiblePentaAlleles - observedAlleles

            alleleCounts = Counter(realMafPenta['allele'])

            for allele, count in alleleCounts.items():
                listOfDicts.append({
                    'allele': allele,
                    'count': count,
                    'penta': penta
                })

            for allele in neverObservedAlleles:
                listOfDicts.append({
                    'allele': allele, 'count': 0, 'penta': penta
                })


    df = pd.DataFrame(listOfDicts)
    return df

In [2]:
poleMafWithPentaContext = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/poleCaseMafWithPentanucleotideContext.maf')

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (36,39,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
poleMafWithPentaContext['pentaChange'] = poleMafWithPentaContext.apply(lambda row: 
                                    mutationSigUtils.create_strand_specific_pentanucleotide_change(row['Ref_Tri.1'], row['Reference_Allele'], row['Tumor_Seq_Allele2'], row['Variant_Type']), axis=1)


**HOTSPOT MUTATIONS**

In [4]:
hotspotContextSummary = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/hotspotContextSummary.tsv')

In [5]:
hotspotContextSummary['pentaChange'] = hotspotContextSummary.apply(lambda row: 
                                    mutationSigUtils.create_strand_specific_pentanucleotide_change(row['Ref_Tri.1'], row['Reference_Allele'], row['Tumor_Seq_Allele2'], row['Variant_Type']), axis=1)
hotspotContextSummary['allele'] = hotspotContextSummary['Hugo_Symbol'] + '_' + hotspotContextSummary['HGVSp_Short']


In [6]:
topTenPolePentas = [x[0] for x in Counter(poleMafWithPentaContext['pentaChange']).most_common(10)]

In [9]:
poleMafHotspots = poleMafWithPentaContext[poleMafWithPentaContext['is-a-hotspot'] == 'Y']

df = summarize_allele_info(poleMafHotspots, hotspotContextSummary, topTenPolePentas)


In [10]:
displayThresh = 3
df['gene'] = df['allele'].apply(lambda x: x.split('_')[0])
displayThresh = 5
df['displayLabel'] = df.apply(lambda row: row['allele'] if row['count'] >= displayThresh else None, axis=1)

In [59]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/hotspotAlleleCounts.tsv', index=False, sep='\t')

In [65]:
df[(df['count'] == 0) & (df['gene'] == 'NF1')]['allele']
#print Counter(df[(df['count'] == 0)]['gene'])

26    NF1_p.R2450Q
62     NF1_p.R461Q
Name: allele, dtype: object

**TRUNCATING MUTATIONS**

In [102]:
def count_number_of_times_allele_is_in_multiplet(maf, allele):
    casesWithAllele = set(maf[maf['allele'] == allele]['Tumor_Sample_Barcode'])
    if len(casesWithAllele) == 0:
        return None
    alleleMaf = maf[maf['Tumor_Sample_Barcode'].isin(casesWithAllele)]
    gene = allele.split('_')[0]
    otherOncogenicGeneAlleles = alleleMaf[(alleleMaf['oncogenic'].notnull()) & (alleleMaf['Hugo_Symbol'] == gene) &(alleleMaf['allele'] != allele)]
    return (1.0*len(set(otherOncogenicGeneAlleles['Tumor_Sample_Barcode'])))/len(casesWithAllele)

In [177]:
#lambda function that marks whether an allele is part of a multiplet
def is_allele_multiplet(maf, allele, tsb):
    gene = allele.split('_')[0]
    if maf[(maf['allele'] != allele) & (maf['oncogenic'].notnull()) & (maf['Hugo_Symbol'] == gene) & (maf['Tumor_Sample_Barcode'] == tsb)].shape[0] > 0:
        return 1
    else:
        return 0

In [71]:
truncatingContextSummary = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/truncatingContextSummary.tsv')

In [72]:
truncatingContextSummary['pentaChange'] = truncatingContextSummary.apply(lambda row: 
                                    mutationSigUtils.create_strand_specific_pentanucleotide_change(row['Ref_Tri.1'], row['Reference_Allele'], row['Tumor_Seq_Allele2'], row['Variant_Type']), axis=1)


In [73]:
poleMafWithPentaContext['allele'] = poleMafWithPentaContext['Hugo_Symbol'] + '_' + poleMafWithPentaContext['HGVSp_Short']
truncatingContextSummary['allele'] = truncatingContextSummary['Hugo_Symbol'] + '_' + truncatingContextSummary['HGVSp_Short']
poleMafWithPentaContext = maf_analysis_utils.fix_mll_genes(poleMafWithPentaContext)


In [75]:
poleMafWithPentaContextTrunc = poleMafWithPentaContext[poleMafWithPentaContext['Consequence'] == 'stop_gained']

dfTruncSum = summarize_allele_info(poleMafWithPentaContextTrunc, truncatingContextSummary, topTenPolePentas)

    

In [76]:
dfTruncSum['gene'] = dfTruncSum['allele'].apply(lambda x: x.split('_')[0])
displayThresh = 5
dfTruncSum['displayLabel'] = dfTruncSum.apply(lambda row: row['gene'] if row['count'] >= displayThresh else None, axis=1)

In [53]:
dfTruncSum.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/alleleCounts.tsv', index=False, sep='\t')

In [77]:
significantAlleles = set(dfTruncSum[dfTruncSum['displayLabel'].notnull()]['allele'])

In [91]:
#RELATED/UNRELATED genes as defined by DNDS
relatedGenes = {
    'Endometrial Cancer': ["AKT1","APC","ARID1A","BCOR","CCND1","CDKN2A.p16INK4a","CTCF","CTNNB1",         
"ESR1","FBXW7","FGFR2","KRAS","NRAS","PIK3CA",
                           "PIK3R1","PPP2R1A","PTEN","RB1","RRAS2","SOX17","SPOP","TP53"],
    'Colorectal Cancer':["AMER1","APC","ARID1A","ASXL1","ATM","B2M","BRAF","CDKN2A.p16INK4a","CTNNB1","ELF3","EPHA3","ERBB3","FBXW7","JUN","KRAS","NRAS","PIK3CA","PIK3R1","PTEN","RBM10","RNF43","SMAD2","SMAD3","SMAD4","SOX9","TCF7L2","TP53"],
    'Glioma': ["ARID1A","ARID2","ATRX","BCOR","BRAF","CBL","CDKN1B","CDKN2A.p14arf","CDKN2A.p16INK4a","CDKN2C","CIC","DNMT3A","EGFR","FUBP1","H3F3A","IDH1","KRAS","NF1","NOTCH1","PDGFRA","PIK3CA","PIK3R1","PPM1D","PTEN","PTPN11","RB1","SETD2","STAG2","TP53"]
}

In [93]:
cohortRelatedGenes = set(relatedGenes['Endometrial Cancer']) | set(relatedGenes['Colorectal Cancer'])

In [99]:
recurrentAlleleThresh = 5
tumorSuppresors = set(['ERRFI1', 'ASXL2', 'PMAIP1', 'ACTG1', 'SUFU', 'FBXO11', 'MEN1', 'FAM58A', 'B2M', 'RB1', 'DUSP22', 'SESN1', 'GPS2', 'RAD51D', 'SMG1', 'CDC73', 'MAP3K1', 'SMARCB1', 'INPP4B', 'PARK2', 'SMAD4', 'CBFB', 'CDH1', 'PPP6C', 'SETDB1', 'SETDB2', 'NF2', 'CDKN2B', 'CDKN2C', 'CDKN2A', 'DDX3X', 'PIK3R1', 'BARD1', 'PDS5B', 'KLF4', 'SPRED1', 'VHL', 'SMAD2', 'PMS1', 'PMS2', 'SETD2', 'GATA3', 'TBL1XR1', 'MUTYH', 'SOCS1', 'FAM175A', 'ROBO1', 'ARID1B', 'ARID1A', 'TCF7L2', 'STK11', 'FOXA1', 'PTEN', 'FAT1', 'FAS', 'CYLD', 'MAX', 'SH2D1A', 'APC', 'NTHL1', 'CTCF', 'KDM5C', 'KMT2C', 'ZFHX3', 'FOXP1', 'PIGA', 'CDKN1B', 'CDKN1A', 'FUBP1', 'MSH2', 'ID3', 'TNFRSF14', 'TRAF3', 'EP400', 'BRIP1', 'ARID4A', 'ARID4B', 'XRCC2', 'DAXX', 'SDHAF2', 'ASXL1', 'AMER1', 'RASA1', 'EGR1', 'MST1', 'SOX17', 'RUNX1', 'PIK3R3', 'NCOR1', 'NF1', 'JAK1', 'PTPRD', 'CHEK2', 'CHEK1', 'SMC1A', 'TMEM127', 'STAG1', 'RAD51', 'TCF3', 'STAG2', 'ARID2', 'RAD50', 'RNF43', 'PARP1', 'BLM', 'CUX1', 'RECQL', 'RAD21', 'PTPN2', 'PTPN1', 'SLX4', 'INHA', 'PAX5', 'IRF1', 'TP53', 'HLA-A', 'IRF8', 'CBL', 'TOP1', 'SHQ1', 'PRDM1', 'NSD1', 'ATXN2', 'CREBBP', 'HDAC4', 'SESN2', 'PPP2R1A', 'EPHA7', 'ATM', 'EPHA3', 'POT1', 'SMAD3', 'MOB3B', 'TBX3', 'POLE', 'ATR', 'FANCD2', 'FH', 'BCORL1', 'SOX9', 'IKZF3', 'TSC1', 'TP63', 'MRE11A', 'SDHC', 'BTG1', 'POLD1', 'CIITA', 'SMC3', 'SAMHD1', 'RTEL1', 'ECT2L', 'PIK3R2', 'CRBN', 'FANCC', 'NBN', 'FANCA', 'HLA-B', 'RECQL4', 'DUSP4', 'ERCC2', 'FBXW7', 'TGFBR2', 'TGFBR1', 'MSH3', 'RBM15', 'TET1', 'TET3', 'SESN3', 'MGA', 'LTB', 'FOXL2', 'SH2B3', 'BCOR', 'HIST1H1D', 'ATRX', 'EP300', 'RAD51C', 'RAD51B', 'HIST1H1B', 'TNFAIP3', 'DICER1', 'ARID5B', 'LATS2', 'FOXO1', 'KEAP1', 'EZH2', 'SP140', 'NKX3-1', 'PBRM1', 'PALB2', 'CIC', 'BRCA1', 'DTX1', 'FLCN', 'SPEN', 'CD58', 'ERCC3', 'ERCC4', 'MSH6', 'BCL11B', 'BMPR1A', 'ERF', 'BRCA2', 'NOTCH2', 'EED', 'MITF', 'ELF3', 'SMARCA4', 'BBC3', 'ANKRD11', 'CEBPA', 'BCL2L11', 'AXIN2', 'AXIN1', 'CDK12', 'ESCO2', 'MLH1', 'SDHB', 'MED12', 'HNF1A', 'RYBP', 'ATP6V1B2', 'DNMT3B', 'KMT2B', 'KMT2A', 'DNMT3A', 'NFKBIA', 'TRAF5', 'KMT2D', 'SPOP', 'RBM10', 'P2RY8', 'TP53BP1', 'TSC2', 'KDM6A', 'EPCAM', 'PHOX2B', 'NPM1', 'BCL10', 'LATS1', 'HOXB13', 'ARID3A', 'PTPRT', 'PTPRS', 'INPPL1', 'NOTCH4', 'TET2', 'NOTCH1', 'CASP8', 'NOTCH3', 'GRIN2A', 'MAP2K4', 'WT1', 'BACH2', 'SDHA', 'BAP1', 'PTCH1', 'SDHD'])
poleMafTruncatingTSG = poleMafWithPentaContextTrunc[poleMafWithPentaContextTrunc['Hugo_Symbol'].isin(tumorSuppresors)]

recurrentTruncatingAlleles = [allele for allele, count in Counter(poleMafWithPentaContextTrunc['allele']).items() if count > recurrentAlleleThresh]
notRecurrentTruncatingAlleles = set(poleMafTruncatingTSG['allele']) - set(recurrentTruncatingAlleles)

In [ ]:
#recurrentTruncatingAlleles
notRecurrentTruncatingAlleles

In [157]:
listOfDicts = []

#We get averages which we will use for our ordering val
alleleFracDict = {}
for allele in significantAlleles:
    gene = allele.split('_')[0]
    fracCasesWithMultiplet = count_number_of_times_allele_is_in_multiplet(poleMafTruncatingTSG, allele)
    alleleFracDict[allele] = fracCasesWithMultiplet
    #geneClass = 'unrelated'
    #if gene in cohortRelatedGenes:
    #    geneClass = 'related'
    #listOfDicts.append({'gene': gene, 'frac': fracCasesWithMultiplet, 
    #            'allele': allele, 'geneClass': geneClass})

#ABOVE IS A DEPRECATED WAY OF DOING THINGS
                
#do a second iteration where we get summary figures for the recurrent and non recurrent alleles

"""relatedVals = []
unrelatedVals = [] 
for allele in notRecurrentTruncatingAlleles:
    gene = allele.split('_')[0]
    fracCasesWithMultiplet = count_number_of_times_allele_is_in_multiplet(poleMafTruncatingTSG, allele)
    if gene in cohortRelatedGenes:
        relatedVals.append(fracCasesWithMultiplet)
    else:
        unrelatedVals.append(fracCasesWithMultiplet)
"""

#alleleFracDict['Average other \nrelated gene alleles'] :  np.nanmean(relatedVals)
#alleleFracDict['Average other \nunrelated gene alleles'] :  np.nanmean(relatedVals)
     
#listOfDicts.append({
    #'Hugo_Symbol': 'Average other \nrelated gene alleles', 'frac': np.nanmean(relatedVals), 
 #               'allele':'Average other \nrelated gene alleles', 'geneClass': 'related_average'})
#listOfDicts.append({
    #'Hugo_Symbol': 'Average other \nunrelated gene alleles', 'frac': np.nanmean(unrelatedVals), 
    #            'allele': 'Average other \nunrelated gene alleles', 'geneClass': 'unrelated_average'})


"relatedVals = []\nunrelatedVals = [] \nfor allele in notRecurrentTruncatingAlleles:\n    gene = allele.split('_')[0]\n    fracCasesWithMultiplet = count_number_of_times_allele_is_in_multiplet(poleMafTruncatingTSG, allele)\n    if gene in cohortRelatedGenes:\n        relatedVals.append(fracCasesWithMultiplet)\n    else:\n        unrelatedVals.append(fracCasesWithMultiplet)\n"

In [191]:
#add columns to the maf so R can plot it with error bars
poleMafTruncatingTSG['displayAllele'] = poleMafTruncatingTSG['allele'].apply(lambda x: x if x in significantAlleles
                    else 'Average other \nrelated gene alleles' if allele.split('_')[0] in cohortRelatedGenes
                    else 'Average other \nunrelated gene alleles')
poleMafTruncatingTSG['multipletPresentInCase'] = poleMafTruncatingTSG.apply(lambda row:
                is_allele_multiplet(poleMafTruncatingTSG, row['allele'], row['Tumor_Sample_Barcode']), axis=1)
poleMafTruncatingTSG['geneClass'] = poleMafTruncatingTSG['Hugo_Symbol'].apply(lambda x: 'related' if x in cohortRelatedGenes else 'unrelated')

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

In [192]:
#poleMafTruncatingTSG.columns.values
df = poleMafTruncatingTSG[['Hugo_Symbol', 'allele', 'multipletPresentInCase', 'displayAllele', 'geneClass']]

In [193]:
df['frac'] = df['allele'].apply(lambda x: alleleFracDict[x] if x in alleleFracDict else 0)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [194]:
#order the plot
df['orderingVal'] = df.apply(lambda row: -.01 if row['displayAllele'] == 'Average other \nrelated gene alleles'
                             else 1.01 if row['displayAllele'] == 'Average other \nunrelated gene alleles'
                            else 1- row['frac'] if row['geneClass'] == 'related'
                            else 2.01 -row['frac'], axis=1)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [196]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/stopGainAlleleRecurrenceAndDoubles.tsv', index=False, sep='\t')

In [185]:
poleMafTruncatingTSG[poleMafTruncatingTSG['allele']== 'APC_p.R2204*']['geneClass']

800      unrelated
4002     unrelated
9995     unrelated
10804    unrelated
10962    unrelated
11187    unrelated
Name: geneClass, dtype: object

**HOTSPOT mutations never seen**

In [31]:
def assign_hotspot_freq_dict(df):
    d = {}
    for index, row in df.iterrows():
        
        refAminoAcid = row['ref']
        gene = row['Hugo_Symbol']
        position = row['Amino_Acid_Position']
        for entry in row['Var_AA'].split('|'):
            fullAltName = ''
            altAminoAcid, count = entry.split(':')
            fullAltName = gene + ':' + refAminoAcid + position + altAminoAcid
            d[fullAltName] = float(count)
    return d


In [12]:
hotspotsNeverSeen = set(df[df['count'] < 1]['allele'])

In [32]:
hotspotsDf = pd.read_table(pathPrefix + '/home/gavrilae/snp_output_final_pancan.txt')
hotspotIncidenceD = assign_hotspot_freq_dict(hotspotsDf)

In [42]:
for hotspot in hotspotsNeverSeen:
    #ERRFI1_p.K116N
    adjHotspot = re.sub('_p.', ':', hotspot)
    if adjHotspot in hotspotIncidenceD:
        print adjHotspot, hotspotIncidenceD[adjHotspot]
    else:
        print adjHotspot, 0

MED12:L1224I 1.0
ERRFI1:K116N 0
CARD11:K215N 0
CTNNB1:S37Y 17.0
PBRM1:R710Q 0
RARA:F286L 1.0
SUZ12:R101Q 0
GATA3:R366Q 7.0
EP300:Q1455H 8.0
PTPN11:E69D 2.0
NOTCH2:R2400Q 0
KMT2A:R1264Q 0
TP53:F113L 1.0
GATA3:R364I 0
NOTCH1:F853L 0
SETD2:S1572L 0
ERCC2:D609N 0
SF3B1:K666N 19.0
IKZF1:E304K 15.0
MTOR:F1888L 7.0
PPP6C:S270L 0
TP63:R379C 23.0
RB1:R787Q 2.0
PIK3R2:D557N 2.0
STAG2:R216Q 3.0
ASXL2:R357Q 1.0
SDHAF2:S10L 5.0
NF1:R461Q 2.0
CRLF2:S128L 12.0
MAP2K1:Q56H 1.0
CDKN2A:S12L 4.0
SMAD4:R445Q 2.0
HIST1H3H:K37N 0
RHOA:E40D 2.0
TP53:K132N 61.0
NFE2L2:L30I 0
PMS2:K651N 0
ARID1A:R2158Q 3.0
CTCF:R448Q 1.0
PIK3CA:R93W 36.0
NF1:R2450Q 1.0
CDH1:R63Q 0
CCND1:E9K 0
ERBB4:K1223N 0
TP53:L32M 0
HIST1H3C:K37N 0
TP53:L194I 0
CTNNB1:S45Y 11.0
